# Bayesian Data Analysis

Recently I have been reading up a lot on bayesian statistics. So I thought it might be fun to apply some bayesian analysis on my own fitbit data to answer a few questions I have: 

- Have my sleep improved (longer and more consistent) over time?
- Has it improved since I joined Fuel?
- Has giving up coffee helped at all?
- Do I tend to get better sleep when I exercise more consistently?
- Does a consistent bed time help lower variability in sleep and increase sleep quality?

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../src/')
import plotutils
import fitbitqueries as fbq

In [3]:
import fitbit
import os
import getpass
import numpy as np
import datetime
import pandas as pd
import matplotlib.pyplot as plt
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

## Connect to fitbit API

In [4]:
os.environ['client_id'] = '22BC6L'
os.environ['client_secret'] = 'e8348b3b97bba4db4fb142928cedfc31'

In [5]:
!/Users/hasannagib/opt/anaconda3/envs/fitbit/bin/python ../src/generate_tokens.py $client_id $client_secret

[17/Nov/2019:12:20:12] ENGINE Listening for SIGTERM.
[17/Nov/2019:12:20:12] ENGINE Listening for SIGHUP.
[17/Nov/2019:12:20:12] ENGINE Listening for SIGUSR1.
[17/Nov/2019:12:20:12] ENGINE Bus STARTING
CherryPy Checker:
The Application mounted at '' has an empty config.

[17/Nov/2019:12:20:12] ENGINE Started monitor thread 'Autoreloader'.
[17/Nov/2019:12:20:12] ENGINE Serving on http://127.0.0.1:8080
[17/Nov/2019:12:20:12] ENGINE Bus STARTED
127.0.0.1 - - [17/Nov/2019:12:20:14] "GET /?code=757b00e2ec310daedaf159bd79a01b70c2a91498&state=cG8RDrrmhqMqIDJCfk1My5PzK6zkU9 HTTP/1.1" 200 122 "" "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.97 Safari/537.36"
[17/Nov/2019:12:20:15] ENGINE Bus STOPPING
[17/Nov/2019:12:20:15] ENGINE HTTP Server cherrypy._cpwsgi_server.CPWSGIServer(('127.0.0.1', 8080)) shut down
[17/Nov/2019:12:20:15] ENGINE Stopped thread 'Autoreloader'.
[17/Nov/2019:12:20:15] ENGINE Bus STOPPED
[17/Nov/2019:12:20:15] ENGIN

In [6]:
with open('../data/access_token.txt','r') as f:
    os.environ['fitbit_access_token'] = f.read()

with open('../data/refresh_token.txt','r') as f:
    os.environ['fitbit_refresh_token'] = f.read()
    
auth_client = fitbit.Fitbit(
    os.environ['client_id'], 
    os.environ['client_secret'],
    os.environ['fitbit_access_token'],
    os.environ['fitbit_refresh_token']
)

## Query sleep data over a date range

In [48]:
df, sleep_raw, sleep_parsed = fbq.get_sleep_data(auth_client, '2018-09-01', pd.datetime.today())

In [70]:
df_plot = df.groupby(pd.Grouper(key='dateOfSleep', freq='W')).mean().query('minutesAsleep > 0')

df_plot['Goal'] = 7.5*60

plotutils.plot_ts(
    df_plot, 
    ys=['minutesAsleep', 'Goal'], 
    hover_vars=['timeInBed', 'index'],
    styles=['-o', '-'],
    plot_height=400,
    plot_width=900
)

In [54]:
count = 0
for d in df.iloc[-1]['minuteData']:
    if d['value'] == '1':
        count += 1

In [56]:
count

463

In [57]:
df.iloc[-1]

awakeCount                                                             1
awakeDuration                                                          1
awakeningsCount                                                        8
dateOfSleep                                          2019-11-17 00:00:00
duration                                                        30300000
efficiency                                                            92
endTime                                          2019-11-17T04:41:30.000
logId                                                        24663870146
minuteData             [{'dateTime': '20:16:30', 'value': '2'}, {'dat...
minutesAfterWakeup                                                     0
minutesAsleep                                                        457
minutesAwake                                                          39
minutesToFallAsleep                                                    0
restlessCount                                      

In [58]:
df.iloc[-1]['minuteData']

[{'dateTime': '20:16:30', 'value': '2'},
 {'dateTime': '20:17:30', 'value': '1'},
 {'dateTime': '20:18:30', 'value': '2'},
 {'dateTime': '20:19:30', 'value': '1'},
 {'dateTime': '20:20:30', 'value': '1'},
 {'dateTime': '20:21:30', 'value': '1'},
 {'dateTime': '20:22:30', 'value': '1'},
 {'dateTime': '20:23:30', 'value': '1'},
 {'dateTime': '20:24:30', 'value': '2'},
 {'dateTime': '20:25:30', 'value': '1'},
 {'dateTime': '20:26:30', 'value': '1'},
 {'dateTime': '20:27:30', 'value': '1'},
 {'dateTime': '20:28:30', 'value': '1'},
 {'dateTime': '20:29:30', 'value': '1'},
 {'dateTime': '20:30:30', 'value': '1'},
 {'dateTime': '20:31:30', 'value': '1'},
 {'dateTime': '20:32:30', 'value': '1'},
 {'dateTime': '20:33:30', 'value': '1'},
 {'dateTime': '20:34:30', 'value': '1'},
 {'dateTime': '20:35:30', 'value': '1'},
 {'dateTime': '20:36:30', 'value': '1'},
 {'dateTime': '20:37:30', 'value': '1'},
 {'dateTime': '20:38:30', 'value': '1'},
 {'dateTime': '20:39:30', 'value': '1'},
 {'dateTime': '2

In [82]:
auth_client.activity_detail(activity_id=26735577422)

HTTPNotFound: Invalid activity id: 26735577422